# Project Supermarket Introduction

## 1) Project Goal

Write a program that simulates customer behavior in a supermarket. The steps towards this goal are:
>- explore the data (includes pandas wrangling)
>- calculate transition probabilities (a 5x5 matrix)
>- implement a customer class
>- run a MCMC simulation for a single customer
>- extend the simulation to multiple customers

### !!Difficult Bonus!!
For the fast groups among you, there is an extra challenge. You can visualize the customer churney through the supermarket! (eg. https://github.com/win-times-win/supermarket_simulation)

### Work in a group!

## 2) Concepts and tools

1. Monte Carlo Methods
2. Markov Chains
3. Object-oriented programming (Classes)
4. Numpy

Let's talk about more about the first two of these concepts

### 2.1) Monte Carlo Methods

A class of computational algorithms that rely on **repeated random sampling** to obtain numerical results. These methods can help us

>1. Estimate expectations if they are not computable
>2. Estimate parameters of (nonlinear) models
>3. Simulate different scenarios

We will use **repeated random sampling** to simulate customer behavior.

The following question arises: **How do we model customer behavior?** To answer this question we introduce the concept of **Markov Chains**.

### 2.2) Markov Chains

- **A Markov Chain is a stochastic process that has the Markov property.**
- "Markov chains, named after Andrey Markov, are mathematical systems that hop from one "state" (a situation or set of values) to another." https://setosa.io/blog/2014/07/26/markov-chains/index.html

#### 2.2.1) Markov Property

A stochastic process has the Markov Property, if **knowledge of the current state is all that is necessary to determine the probability distribution of future states**. The stochastic process is said to be memoryless.

#### 2.2.2) Assumptions

1. Markov Property: Knowledge of the current state is all that is necessary to determine the probability distribution of future states.
2. Finite State Space: `states = []`
3. Time-homogenous: transition probabilities do not change with time

#### 2.2.3) Characteristics

A Markov Chain can be characterized by three elements:

- State space (all possible states)
- Initial state or initial probability distribution
- Transition probabilities

In [1]:
import random
import pandas as pd
import numpy as np

![](./markov_weather.png)

In [2]:
# Let's store that information
weather = ['snow', 'snow', 'sun', 'snow', 'snow', 'sun', 'snow', 'sun', 'sun']
weather

['snow', 'snow', 'sun', 'snow', 'snow', 'sun', 'snow', 'sun', 'sun']

## What are the possible states?

The possible states are sun and snow.
The state space S = ['sun', 'snow']

In [13]:
S = ['snow', 'sun']

## How can we calculate the transition matrix?

The transition matrix $P$ has the element $p_{ij}$, with rows $i$ and columns $j$, such that:

$$
p_{ij} = P(Y_t = y_j | Y_{t-1} = y_i)
$$

For example $p_{0,1} = p_{snow, sun}$ is the probability of a sunny day when it was snowing the day before. 

In [3]:
weather = pd.DataFrame({'day': range(1,len(weather)+1), 'weather': weather})
weather

,day,weather
0,1,snow
1,2,snow
2,3,sun
3,4,snow
4,5,snow
5,6,sun
6,7,snow
7,8,sun
8,9,sun


In [4]:
# Create a column containing the next state
weather['transition'] = weather['weather'].shift(-1)
weather

,day,weather,transition
0,1,snow,snow
1,2,snow,sun
2,3,sun,snow
3,4,snow,snow
4,5,snow,sun
5,6,sun,snow
6,7,snow,sun
7,8,sun,sun
8,9,sun,NaN


In [5]:
# How do we now calculate the empirical transition probabilities?
weather.groupby(['weather', 'transition']).count()

day
weather transition     
snow    snow          2
        sun           3
sun     snow          2
        sun           1

In [11]:
# Use pd.crosstab to calculate the transition probabilites
P = pd.crosstab(weather['weather'], weather['transition'], normalize='index')
P

# P is the transition probability matrix

transition,snow,sun
weather,,
snow,0.400000,0.600000
sun,0.666667,0.333333


## Creating a state diagram with [`pygraphviz`](http://pygraphviz.github.io/documentation/pygraphviz-1.5/)

In [7]:
#pip install pygraphviz

In [8]:
import pygraphviz as pgv

states = ['snow', 'sun']
G = pgv.AGraph(directed=True)
for state_from in states:
    for state_to in states:
        G.add_edge(state_from, state_to, label=np.round(P.loc[state_from, state_to],2))

G.draw('transition.png', prog='dot')

ModuleNotFoundError: No module named 'pygraphviz'

## So we have

$$
P(Y_{t+1}=0| Y_{t}=0) = 0.4
$$
$$
P(Y_{t+1}=0| Y_{t}=1) = 0.67
$$
$$
P(Y_{t+1}=1| Y_{t}=0) = 0.60
$$
$$
P(Y_{t+1}=1| Y_{t}=1) = 0.33
$$

What is the probability of a third day of sun given that the two previous days where sunny as well (Markov property!)?

$$
P(Y_{3}=1|Y_{2}=1, Y_{1}=1) = P(Y_{3}=1|Y_{2}=1) = 0.33
$$

What is the probability of a sunny third day given that the first day was sunny and we know nothing about the second day? 

$$
P(Y_{3}=1|Y_{1}=1) = P(Y_{3}=1| Y_{2}=1)P(Y_{2}=1|Y_{1}=1) + P(Y_{3}=1| Y_{2}=0)P(Y_{2}=0|Y_{1}=1) = 0.33*0.33 + 0.6*0.67 = 0.51
$$

In general we can write:

$$
P(Y_{1+h} = y_{1+h} | Y_{1}=y_{1}) = P(Y_{1}=y_{1})P^h
$$

where $P(Y_{0})$ is the given initial distribution.

In [10]:
# Choose a current state
current_state = [0, 1] # [0, 1] means sunny and [1, 0] means snowy
# We know that it is sunny

In [40]:
P

transition,snow,sun
weather,,
snow,0.400000,0.600000
sun,0.666667,0.333333


In [39]:
P.loc['sun']

transition
snow    0.666667
sun     0.333333
Name: sun, dtype: float64

In [64]:
# What is the probablity for the conditions tomorrow?
np.dot(current_state, P)

array([0.66666667, 0.33333333])

In [81]:
# How can we actually choose, given the current probability distribution, a next stat
np.random.choice(S, p=P.loc['sun'])
# np.random.choice(S, p=np.dot(current_state, P))

# Both solutions are equivalent

'snow'

In [65]:
# What is the probability for the conditions in two days?
np.dot(np.dot(current_state, P), P)

array([0.48888889, 0.51111111])

In [73]:
# Look at the stationary distribution
P.dot(P).dot(P).dot(P).dot(P).dot(P).dot(P).dot(P)

transition,snow,sun
weather,,
snow,0.526328,0.473672
sun,0.526302,0.473698


#### How can we characterize the Markov Chain in the supermarket project?

In [ ]:
# Initial State
# You could randomly draw from all states but the checkout

# State Space - state of all possible states
S = ['dairy', 'drinks', 'spices', 'fruit', 'checkout']

# Transition Probabilites
# Your task to extract them from the data